In [9]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from parse_logs.parse_base import LogDir
from parse_logs.parse_experiment_results import parse_experiment_result
from parse_logs.parse_coordinator_logs import read_design

from parse_logs.parse_coordinator_logs import get_flat_design
from flatten_json import flatten

sns.set(rc={"figure.figsize":(16, 8)})
# parser.root_data_path = '../tests/data'
#trial_run_objects = parser.get_trial_runs(
#    exp_run_code='experiment_2021_04_01_16_20_00_run_1')

LogDir.base_data_path = '../data'
trial_run_objects = list(parse_experiment_result(exec_code='experiment_2021_07_29_15_33_17_run_1'))



ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"
ignoring "trial.log, wrong name format"


In [17]:
no_executor = [trial_run.to_dict() for trial_run in trial_run_objects if not trial_run.executor]
no_executor.sort( key=lambda tr: tr['exec_group']+tr['code'])
ne_df = pd.DataFrame.from_records(no_executor)

ne_df

,exec_group,scenario_id,code,treatment,executor,ttc,failure_time,end_state,total_time_wall_clock,has_failure
0,les-01,36,abaccp,None,None,NaN,NaN,timeout-wall,2700.61,False
1,les-01,51,abcbcb,None,None,NaN,9.48,no-skill,45.05,True
2,les-01,58,acabab,None,None,NaN,9.10,no-skill,36.03,True
3,les-01,78,accbcb,None,None,NaN,11.58,no-skill,40.04,True
4,les-03,14,aabbbb,None,None,NaN,901.00,timeout-sim,1163.10,True
5,les-03,32,ababbp,None,None,NaN,662.85,low-battery,865.82,True
6,les-03,51,abcbcb,None,None,NaN,16.57,no-skill,41.04,True
7,les-03,58,acabab,None,None,NaN,21.88,no-skill,41.06,True
8,les-03,70,acbcab,None,None,NaN,368.87,low-battery,553.53,True
9,les-03,73,accaap,None,None,NaN,NaN,timeout-wall,2700.74,False


In [19]:
ne_df2 = ne_df[ne_df.end_state != 'no-skill']
ne_df2

,exec_group,scenario_id,code,treatment,executor,ttc,failure_time,end_state,total_time_wall_clock,has_failure
0,les-01,36,abaccp,None,None,NaN,NaN,timeout-wall,2700.61,False
4,les-03,14,aabbbb,None,None,NaN,901.00,timeout-sim,1163.10,True
5,les-03,32,ababbp,None,None,NaN,662.85,low-battery,865.82,True
8,les-03,70,acbcab,None,None,NaN,368.87,low-battery,553.53,True
9,les-03,73,accaap,None,None,NaN,NaN,timeout-wall,2700.74,False
11,les-04,1,aaaaap,None,None,409.42,NaN,success,631.61,False
12,les-04,34,abacap,None,None,NaN,NaN,timeout-wall,2700.70,False
14,les-04,57,acaacb,None,None,NaN,NaN,timeout-wall,2700.39,False
17,les-05,1,aaaaab,None,None,NaN,NaN,timeout-wall,2700.41,False
18,les-05,25,aaccap,None,None,NaN,426.18,low-battery,558.54,True


In [20]:
len(ne_df2)

22